In [ ]:
#load raw data set
import pandas as pd


df = pd.read_csv(r"C:\Users\NPG\Desktop\maheswar\churn_prediction\churn-prediction-dashboard\data\raw\WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

# Check the shape of the dataset
print("Shape of the dataset:", df.shape)
# Check overall info and missing values
print(df.info())
print("\nMissing values per column:\n", df.isnull().sum())



# 1. Standardize column names to snake_case
df.columns = (
    df.columns
      .str.strip()                # remove whitespace
      .str.lower()                # lowercase
      .str.replace(' ', '_')      # spaces → underscores
)
print("Columns after standardization:\n", df.columns.tolist())


# 2. Convert TotalCharges (currently object) → numeric
df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')
print("Missing after conversion:", df['totalcharges'].isnull().sum())

# 3. Drop rows with missing TotalCharges
df = df.dropna(subset=['totalcharges'])
print("New shape after dropping:", df.shape)


# Reload the raw or intermediate DataFrame
import pandas as pd
df = pd.read_csv(r"C:\Users\NPG\Desktop\maheswar\churn_prediction\churn-prediction-dashboard\data\raw\WA_Fn-UseC_-Telco-Customer-Churn.csv")

# Standardize columns
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Convert TotalCharges and drop NaNs
df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')
df = df.dropna(subset=['totalcharges'])

#  binary encoding, including gender
df['gender'] = df['gender'].map({'Male': 1, 'Female': 0})


for col in ['partner','dependents','phoneservice','paperlessbilling','churn']:
    df[col] = df[col].map({'Yes': 1, 'No': 0})

# One-hot encode multi-category columns
multi_cat = [
    'multiplelines','internetservice','onlinesecurity','onlinebackup',
    'deviceprotection','techsupport','streamingtv','streamingmovies',
    'contract','paymentmethod'
]
df = pd.get_dummies(df, columns=multi_cat, drop_first=True)


print("Unique values in gender:", df['gender'].unique())
# %%
# Convert any boolean columns (True/False) to integers 1/0
bool_cols = df.select_dtypes(include='bool').columns
for col in bool_cols:
    df[col] = df[col].astype(int)

print("Converted boolean columns to ints:", list(bool_cols))

# Export cleaned DataFrame
import os
os.makedirs("data/cleaned", exist_ok=True)
df.to_csv("data/cleaned/telco_churn_cleaned.csv", index=False)
print("Export done with shape:", df.shape)


# %%
import pandas as pd
df2 = pd.read_csv("data/cleaned/telco_churn_cleaned.csv")
print(df2.dtypes.value_counts())
print(df2.head())

import pandas as pd

# 1. Load your cleaned CSV
df = pd.read_csv("data/cleaned/telco_churn_cleaned.csv")

# 2. Define base columns and their SQL types
base_cols = {
    "customerid": "VARCHAR PRIMARY KEY",
    "gender": "SMALLINT",
    "seniorcitizen": "SMALLINT",
    "partner": "SMALLINT",
    "dependents": "SMALLINT",
    "tenure": "INTEGER",
    "phoneservice": "SMALLINT",
    "paperlessbilling": "SMALLINT",
    "monthlycharges": "NUMERIC(8,2)",
    "totalcharges": "NUMERIC(10,2)",
    "churn": "SMALLINT"
}

# 3. Build the DDL lines
lines = ["DROP TABLE IF EXISTS telco_customers;", "CREATE TABLE telco_customers ("]
for col, sql_type in base_cols.items():
    lines.append(f"  {col} {sql_type},")
for col in df.columns:
    if col not in base_cols:
        # sanitize names for SQL
        safe_col = col.lower().replace(" ", "_").replace("(", "").replace(")", "")
        lines.append(f"  {safe_col} SMALLINT,")
# remove the trailing comma on the last column
lines[-1] = lines[-1].rstrip(",")
lines.append(");")

# 4. Print the DDL
print("\n".join(lines))

import pandas as pd

df = pd.read_csv("data/cleaned/telco_churn_cleaned.csv")
print("Churned count:", df[df['churn'] == 1].shape[0])
